In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib as mp
import random

In [21]:
norm = pd.read_csv('Data/norm_loan_lenders.csv', usecols = ['loan_id','lenders'])

## Exercise 1
Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.

In [2]:
loan_lenders = pd.read_csv('Data/loans_lenders.csv')
loan_lenders.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


In [3]:
loan_lenders['lenders'] = loan_lenders['lenders'].str.split(',\s')

In [101]:
loan_lenders = loan_lenders[:5000]

In [ ]:
%%time
norm = pd.DataFrame(columns=['loan_id','lenders'])  
norm = norm.append([dic for i in [[{'loan_id':loan_lenders['loan_id'][row],'lenders':l} for l in loan_lenders['lenders'][row]] for row in range(len(loan_lenders))] for dic in i],ignore_index = True)

In [ ]:
norm.to_csv('Data/norm_loan_lenders.csv',columns = ['loan_id','lenders'])

### SECONDA SOLUZIONE

In [86]:
def dataframe_from_row (index):
    id_loan = loan_lenders.at[index,'loan_id']
    lenders = loan_lenders.at[index,'lenders']
    rep_id = [id_loan]*len(lenders)
    data = {'loan_id':rep_id,'lender_norm':lenders}
    return data

In [87]:
indexes = len(list(loan_lenders.index)[:5000])

In [99]:
%%time
norm = pd.DataFrame(columns = ['loan_id','lender_norm'])
norm = norm.append([pd.DataFrame.from_dict(dataframe_from_row(index)) for index in range(indexes)],ignore_index = True)

Wall time: 2.94 s


In [95]:
norm

,loan_id,lender_norm
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
135937,128485,xxAnonyMousexx
135938,128485,klassenkasse9334
135939,128485,merle2926
135940,128485,guillermo5352


# Exercise 2
For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time.

In [ ]:
norm

# Exercise 3
Find the lenders that have funded at least twice.

In [ ]:
lenders_twice = norm.groupby('lenders').count() > 1
list(lenders_twice[lenders_twice['loan_id'] == True].index)

# Exercise 4
For each country, compute how many loans have involved that country as borrowers.

# Exercise 5
For each country, compute the overall amount of money borrowed.


In [4]:
loans = pd.read_csv('Data/loans.csv')

In [5]:
money_per_country = loans.groupby('country_name')['funded_amount'].sum()

In [6]:
money_per_country

country_name
Afghanistan        1964650.0
Albania            3992475.0
Armenia           20284675.0
Azerbaijan        14191350.0
Belize              150175.0
                     ...    
Vietnam           23463825.0
Virgin Islands           0.0
Yemen              3340025.0
Zambia             1913250.0
Zimbabwe           5818550.0
Name: funded_amount, Length: 96, dtype: float64

# Exercise 6
Like the previous point, but expressed as a percentage of the overall amount lent.


In [7]:
loans[loans['funded_amount'] != loans['loan_amount']][['funded_amount','loan_amount']]
#Which one is which? I'll pick the lower one!

,funded_amount,loan_amount
22,100.0,225.0
25,225.0,300.0
36,425.0,1000.0
46,275.0,875.0
50,325.0,1200.0
...,...,...
1419525,125.0,225.0
1419538,600.0,3000.0
1419545,725.0,1025.0
1419564,50.0,8750.0


In [8]:
money_per_country_percentage = loans.groupby('country_name')['funded_amount'].sum() \
                             / loans.groupby('country_name')['loan_amount'].sum() * 100
money_per_country_percentage

country_name
Afghanistan        99.832313
Albania            92.689821
Armenia            88.384554
Azerbaijan         95.987216
Belize            100.000000
                     ...    
Vietnam            95.067987
Virgin Islands      0.000000
Yemen              96.980981
Zambia             96.678836
Zimbabwe           99.430524
Length: 96, dtype: float64

# Exercise 7
Like the three previous points, but split for each year (with respect to disburse time).


In [9]:
loans['disburse_year'] = pd.to_datetime(loans['disburse_time']).dt.year

In [10]:
loans_per_nation_year = loans.groupby(['country_name','disburse_year']).count()
loans_per_nation_year

loan_id  loan_name  original_language  \
country_name disburse_year                                          
Afghanistan  2007.0             408        408                408   
             2008.0             370        369                369   
             2009.0             678        614                614   
             2010.0             632        291                291   
             2011.0             247          0                  0   
...                             ...        ...                ...   
Zimbabwe     2013.0             426        388                388   
             2014.0            2078       2029               2029   
             2015.0             600        589                589   
             2016.0             808        787                787   
             2017.0            1079       1078               1078   

                            description  description_translated  \
country_name disburse_year                                        
Afghanistan  2007.0                 408                       0   
             2008.0                 369                       0   
             2009.0                 614                       0   
             2010.0                 291                       0   
             2011.0                   0                       0   
...                                 ...                     ...   
Zimbabwe     2013.0                 388                       0   
             2014.0                2029                    2025   
             2015.0                 589                     589   
             2016.0                 787                     787   
             2017.0                1078                    1078   

                            funded_amount  loan_amount  status  activity_name  \
country_name disburse_year                                                      
Afghanistan  2007.0                   408          408     408            408   
             2008.0                   370          370     370            370   
             2009.0                   678          678     678            678   
             2010.0                   632          632     632            632   
             2011.0                   247          247     247            247   
...                                   ...          ...     ...            ...   
Zimbabwe     2013.0                   426          426     426            426   
             2014.0                  2078         2078    2078           2078   
             2015.0                   600          600     600            600   
             2016.0                   808          808     808            808   
             2017.0                  1079         1079    1079           1079   

                            sector_name  ...  raised_time  lender_term  \
country_name disburse_year               ...                             
Afghanistan  2007.0                 408  ...          408          408   
             2008.0                 370  ...          370          370   
             2009.0                 678  ...          674          678   
             2010.0                 632  ...          632          632   
             2011.0                 247  ...          247          247   
...                                 ...  ...          ...          ...   
Zimbabwe     2013.0                 426  ...          426          426   
             2014.0                2078  ...         2073         2078   
             2015.0                 600  ...          599          600   
             2016.0                 808  ...          807          808   
             2017.0                1079  ...         1047         1079   

                            num_lenders_total  num_journal_entries  \
country_name disburse_year                                           
Afghanistan  2007.0                       408                  408   
             2008.0            

In [11]:
money_per_country_year = loans.groupby(['country_name','disburse_year'])['funded_amount'].sum()
money_per_country_year

country_name  disburse_year
Afghanistan   2007.0            194975.0
              2008.0            365375.0
              2009.0            581825.0
              2010.0            563350.0
              2011.0            245125.0
                                 ...    
Zimbabwe      2013.0            678525.0
              2014.0           1307975.0
              2015.0            723150.0
              2016.0            787550.0
              2017.0           1214125.0
Name: funded_amount, Length: 748, dtype: float64

In [12]:
money_per_country_percentage_year = loans.groupby(['country_name','disburse_year'])['funded_amount'].sum() \
                                  / loans.groupby(['country_name','disburse_year'])['loan_amount'].sum() * 100
money_per_country_percentage_year

country_name  disburse_year
Afghanistan   2007.0           100.000000
              2008.0           100.000000
              2009.0            99.436018
              2010.0           100.000000
              2011.0           100.000000
                                  ...    
Zimbabwe      2013.0           100.000000
              2014.0            99.725521
              2015.0            99.934358
              2016.0            99.866853
              2017.0            98.103184
Length: 748, dtype: float64

# Exercise 8
For each lender, compute the overall amount of money lent.

In [5]:
temp = loans[['loan_id','loan_amount']].dropna()

In [6]:
loan_lenders['count'] = loan_lenders.apply(lambda row: len(row['lenders']),axis=1)
loan_lenders

,loan_id,lenders,count
0,483693,"[muc888, sam4326, camaran3922, lachheb1865, re...",40
1,483738,"[muc888, nora3555, williammanashi, barbara5610...",15
2,485000,"[muc888, terrystl, richardandsusan8352, sherri...",15
3,486087,"[muc888, james5068, rudi5955, daniel9859, don9...",13
4,534428,"[muc888, niki3008, teresa9174, mike4896, david...",19
...,...,...,...
1387427,678999,"[michael43411218, carol5987, gooddogg1, chris4...",10
1387428,1207353,"[rjhoward1986, jeffrey6870, trolltech4460, ely...",5
1387429,1206220,"[vicky7746, gooddogg1, fairspirit, craig972996...",44
1387430,1206425,"[rich6705, sergiiy9766, angela7509, barbara561...",8


In [7]:
merged = pd.merge(temp,loan_lenders,on='loan_id')
merged['funded_divided'] = merged['loan_amount'] / merged['count']
merged

,loan_id,loan_amount,lenders,count,funded_divided
0,657307,125.0,"[spencer5657, matthew8640, larry71496105]",3,41.666667
1,657259,400.0,"[ltr, andrew5306, dana9865, WHYu, robert978452...",7,57.142857
2,658010,400.0,"[kathy3100, omar7511, amirali5409, bingo, geni...",14,28.571429
3,659347,625.0,"[jasonsamfield, mikaela2498, tim1351, rifath92...",17,36.764706
4,656933,425.0,"[john86857365, gooddogg1, daniel8458, anjae514...",14,30.357143
...,...,...,...,...,...
1387423,988180,400.0,"[joyce5432, douglas5957, michelle27516947, wak...",13,30.769231
1387424,988213,300.0,"[dqqpkh5136, gerald4889, peter2548, tatjanacha...",11,27.272727
1387425,989109,2425.0,"[ulrike5921, oakviewroberts, kenmwong, trudy32...",69,35.144928
1387426,989143,100.0,"[jack6206, david85674927]",2,50.000000


In [43]:
funded_by_lender = {}
for i in range(len(merged)):
    for l in merged['lenders'][i]:
        funded_by_lender[l] = funded_by_lender.get(l, 0) + merged['funded_divided'][i]

In [ ]:
funded_by_lender

# Exercise 9
For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

In [46]:
lenders = pd.read_csv('Data\lenders.csv')
lenders

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
2349169,janet7309,Janet,NaN,NaN,NaN,1342097163,NaN,NaN,NaN,NaN,0
2349170,pj4198,NaN,NaN,NaN,NaN,1342097515,NaN,NaN,NaN,NaN,0
2349171,maria2141,Maria,NaN,NaN,US,1342099723,NaN,NaN,2.0,NaN,0
2349172,simone9846,Simone,NaN,NaN,NaN,1342100213,NaN,NaN,NaN,NaN,0


In [ ]:
received_country = loans.groupby('country_code').sum()['loan_amount']
received_country

In [44]:
funded_by_lenders_df = pd.DataFrame.from_dict(funded_by_lender,orient='index',columns = ['funded_amount'])
funded_by_lenders_df['lenders'] = funded_by_lenders_df.index

In [ ]:
funded_by_lenders_df

In [ ]:
lenders_country = pd.merge(lenders[['permanent_name','country_code']],funded_by_lenders_df,left_on = 'permanent_name',right_on = 'lenders')
lenders_country.dropna().head()

In [ ]:
a = (lenders_country.dropna().groupby('country_code').count()/len(lenders_country.dropna()))['lenders']

In [ ]:
c = list(a.index)
p = list(a.values)

In [ ]:
x = pd.Series(np.random.choice(c, len(lenders_country[lenders_country['country_code'].isnull()]), True, p))

In [ ]:
keys = list(lenders_country[lenders_country['country_code'].isnull()].index)
values = list(x)

In [ ]:
lenders_country['country_code'] = lenders_country['country_code'].fillna(dict(zip(keys,values)))

In [ ]:
borrowed_country = lenders_country.groupby('country_code').sum()['funded_amount']
borrowed_country

In [ ]:
diff_borrow_lent = received_country - borrowed_country
diff_borrow_lent.dropna()

# Exercise 10
Which country has the highest ratio between the difference computed at the previous point and the population?

In [9]:
countries = pd.read_csv('Data\country_stats.csv')
countries

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.5,0.578824,71.985,10.178706,5.241577,3341.490722,Bangladesh
...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Somalia,SO,SOM,Africa,Eastern Africa,14742523,NaN,NaN,NaN,NaN,NaN,NaN,Somalia
170,Central African Republic,CF,CAF,Africa,Middle Africa,4659080,NaN,0.352440,51.458,7.098980,4.230000,587.473961,Central African Republic
171,Samoa,WS,WSM,Oceania,Polynesia,196440,NaN,0.702000,NaN,NaN,NaN,NaN,Samoa
172,Palestine,PS,PS,Asia,Western Asia,4920724,NaN,0.677000,NaN,NaN,NaN,NaN,Palestine


In [ ]:
diff_df = diff_borrow_lent.to_frame()
diff_df.columns = ['diff']

In [ ]:
pop_diff = pd.merge(diff_df,countries[['country_name','country_code','population','population_below_poverty_line']],left_index = True,right_on = 'country_code')
pop_diff['ratio'] = pop_diff['diff']/pop_diff['population']
pop_diff.loc[pop_diff['ratio'].idxmax()]

# Exercise 11
Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

In [ ]:
countries.sort_values(by =['country_code'])[['country_code','population']]

In [ ]:
pop_diff['ratio_not_poor'] = pop_diff['diff']/((100-pop_diff['population_below_poverty_line'])*pop_diff['population']/100)
pop_diff.loc[pop_diff['ratio_not_poor'].idxmax()]

# Exercise 12
For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

# Exercise 13
For each value of repayment_interval, add a new column to the lenders dataframe that contains the total amount of money corresponding to loans in such state

In [20]:
loans['repayment_interval'].unique()

array(['irregular', 'monthly', 'bullet', 'weekly'], dtype=object)

In [11]:
loans.groupby(['repayment_interval','loan_id']).sum()['loan_amount']

repayment_interval  loan_id
bullet              84         500.0
                    85         500.0
                    86         500.0
                    88         300.0
                    89         500.0
                               ...  
weekly              1090030    325.0
                    1090039    325.0
                    1090040    325.0
                    1090041    125.0
                    1090099    325.0
Name: loan_amount, Length: 1419607, dtype: float64

In [10]:
merged_status = pd.merge(merged,loans[['loan_id','repayment_interval']],on='loan_id')
merged_status

,loan_id,loan_amount,lenders,count,funded_divided,repayment_interval
0,657307,125.0,"[spencer5657, matthew8640, larry71496105]",3,41.666667,irregular
1,657259,400.0,"[ltr, andrew5306, dana9865, WHYu, robert978452...",7,57.142857,monthly
2,658010,400.0,"[kathy3100, omar7511, amirali5409, bingo, geni...",14,28.571429,monthly
3,659347,625.0,"[jasonsamfield, mikaela2498, tim1351, rifath92...",17,36.764706,monthly
4,656933,425.0,"[john86857365, gooddogg1, daniel8458, anjae514...",14,30.357143,bullet
...,...,...,...,...,...,...
1387423,988180,400.0,"[joyce5432, douglas5957, michelle27516947, wak...",13,30.769231,monthly
1387424,988213,300.0,"[dqqpkh5136, gerald4889, peter2548, tatjanacha...",11,27.272727,irregular
1387425,989109,2425.0,"[ulrike5921, oakviewroberts, kenmwong, trudy32...",69,35.144928,irregular
1387426,989143,100.0,"[jack6206, david85674927]",2,50.000000,irregular


In [116]:
possible_repayments = list(loans['repayment_interval'].unique())
for interval in possible_repayments:
    lenders[interval] = 0

In [111]:
lenders = lenders.set_index('permanent_name')

In [ ]:
%%time
for row in merged_status.index[:1000]:
    for l in merged_status.at[row,'lenders']:
        status = merged_status.at[row,'repayment_interval']
        lenders.ix[l,status] += merged_status.at[row,'funded_divided']        

In [130]:
norm[:5000].groupby('lenders')['loan_id'].apply(list)

Wall time: 429 ms


lenders
0545998               [319643]
1961          [356152, 356152]
25perday              [573696]
3beditions            [391924]
9678                  [524323]
                    ...       
zen7998               [573696]
zim7148               [315025]
zoe6003               [396664]
zoec                  [494337]
zsazsa8211            [308897]
Name: loan_id, Length: 4391, dtype: object

In [135]:
len(norm)/5000*429e-3/60

40.46032132999999

# Exercise 14
What is the occupation with the highest average amount of money lent (the average must be computed over all lenders with a given occupation)?

In [42]:
lenders

,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited,irregular,monthly,bullet,weekly
permanent_name,,,,,,,,,,,,,,
qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0,0,0,0,0
reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0,0,0,0,0
mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0,0,0,0,0
andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0,0,0,0,0
nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
janet7309,Janet,NaN,NaN,NaN,1342097163,NaN,NaN,NaN,NaN,0,0,0,0,0
pj4198,NaN,NaN,NaN,NaN,1342097515,NaN,NaN,NaN,NaN,0,0,0,0,0
maria2141,Maria,NaN,NaN,US,1342099723,NaN,NaN,2.0,NaN,0,0,0,0,0


In [47]:
lenders_occupation = pd.merge(lenders[['permanent_name','occupation']],funded_by_lenders_df,left_on = 'permanent_name',right_on = 'lenders')
lenders_occupation.dropna().head()

,permanent_name,occupation,funded_amount,lenders
31,vikas1098,Software Engineer,91.666667,vikas1098
177,kumari2781,Software Engineer,63.873626,kumari2781
390,javier7867,Technology Consultant,457.002956,javier7867
945,jens1183,IT Consultant,59.343605,jens1183
1099,pankaj1930,doctor,137.546380,pankaj1930


In [54]:
lenders_occupation[lenders_occupation['occupation'] == lenders_occupation.groupby('occupation').mean()['funded_amount'].idxmax()]

,permanent_name,occupation,funded_amount,lenders
231277,gooddogg1,www.linkedin.com/in/peacekeeper,8.642502e+06,gooddogg1


# Exercise 15
Cluster the loans according to the year-month of disburse time.

In [68]:
loans['disburse_time'] = pd.to_datetime(loans['disburse_time'])
loans['month_year'] = loans['disburse_time'].dt.year.astype(str) + '-' + loans['disburse_time'].dt.month.astype(str)

In [70]:
loans.groupby('month_year').count()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
month_year,,,,,,,,,,,,,,,,,,,,,
2005.0-4.0,203,187,187,187,0,203,203,203,203,203,...,203,203,203,203,203,2,187,187,203,203
2006.0-10.0,146,112,112,112,0,146,146,146,146,146,...,145,146,146,146,146,0,112,112,146,146
2006.0-11.0,744,525,525,525,0,744,744,744,744,744,...,744,744,744,744,744,1,525,525,744,744
2006.0-12.0,804,609,609,609,0,804,804,804,804,804,...,804,804,804,804,804,0,609,609,804,804
2006.0-3.0,1,0,0,0,0,1,1,1,1,1,...,0,1,1,1,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017.0-9.0,17336,17286,17293,17293,17293,17336,17336,17336,17336,17336,...,16001,17336,17336,17336,17336,14856,17293,17293,17336,17336
2018.0-1.0,339,323,334,334,334,339,339,339,339,339,...,277,339,339,339,339,270,334,334,339,339
2018.0-2.0,560,560,560,560,560,560,560,560,560,560,...,307,560,560,560,560,551,560,560,560,560


# Exercise 16
For each country, compute its overall GDP, by multiplying the per capita GDP with its population.

In [74]:
countries['overall_gni'] = countries['gni']*countries['population']
countries.head()

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name,overall_gni
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India,7.584413e+12
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria,1.038975e+12
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico,2.116096e+12
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan,9.912214e+11
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.5,0.578824,71.985,10.178706,5.241577,3341.490722,Bangladesh,5.502424e+11


# Exercise 17
Find the country with the highest rate of irregular repayment interval.

In [94]:
loans[loans['repayment_interval'] == 'irregular'].groupby('country_name').count().idxmax()['country_code']

'Philippines'

# Exercise 18
Find the country with the highest fraction of loaned amount with irregular repayment interval.

In [110]:
irregular_fraction = loans[loans['repayment_interval'] == 'irregular'].groupby('country_name').sum()['loan_amount'] \
                                                               / loans.groupby('country_name').sum()['loan_amount']
irregular_fraction.idxmax()

'Bhutan'